In [449]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1. Data Quality Assurance

In [450]:
#load the SAT data
dSAT = pd.read_csv('SAT.txt',sep='\t')
dSAT.head()

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
0,1,110.0,10,1320,461,429,430,23.5,88
1,1,120.0,10,1502,515,506,481,41.7,660
2,1,590.0,25,1418,486,468,464,31.1,255
3,1,1310.0,5,1479,504,490,485,37.8,450
4,1,1790.0,40,1477,508,486,483,35.7,235


In [451]:
#EDA
dSAT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
CO_CODE              400 non-null int64
DIST_CODE            396 non-null float64
SCH_CODE             400 non-null int64
TOTAL                400 non-null int64
MATHEMATICS          400 non-null int64
CRITICAL_READING     400 non-null int64
WRITING              400 non-null int64
SAT_1550             400 non-null float64
N_STUDENTS_SCORED    400 non-null int64
dtypes: float64(2), int64(7)
memory usage: 28.2 KB


In [452]:
#There are 4 nulls in the District Code field
dSAT[dSAT.DIST_CODE.isnull()]
#Perhaps there are other data entries with the same school and county code that I can use to fill in District Code

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
106,11,NaN,20,1199,423,386,390,5.5,361
107,11,NaN,30,1380,472,454,454,28.5,32
108,11,NaN,50,1363,464,453,446,19.4,254
109,11,NaN,50,1411,490,464,457,28.1,127


In [453]:
dSAT[dSAT.CO_CODE==11]
#All entries with County Code 11 have null Dist_Code

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
106,11,NaN,20,1199,423,386,390,5.5,361
107,11,NaN,30,1380,472,454,454,28.5,32
108,11,NaN,50,1363,464,453,446,19.4,254
109,11,NaN,50,1411,490,464,457,28.1,127


In [454]:
#If I knew which state this data was from I could just look up the district code using the County and School Code
#For the time being, I will just assign these row a DISTRICT CODE OF 0 as a placeholder.
dSAT.loc[dSAT.CO_CODE==11,'DIST_CODE'] = 0

In [455]:
#Now to change the District Codes from float to int so they are consistent with County and School Code.
dSAT.loc[:,'DIST_CODE'] = dSAT.DIST_CODE.astype('int64')
dSAT.head()

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
0,1,110,10,1320,461,429,430,23.5,88
1,1,120,10,1502,515,506,481,41.7,660
2,1,590,25,1418,486,468,464,31.1,255
3,1,1310,5,1479,504,490,485,37.8,450
4,1,1790,40,1477,508,486,483,35.7,235


In [456]:
#Check for duplicate rows
dSAT[dSAT.duplicated()]

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
382,41,280,20,1527,509,510,508,46.6,381
383,41,1870,50,1513,519,498,496,44.8,306
384,41,3675,50,1502,514,503,485,44.3,282
385,41,4100,50,1543,529,513,501,44.1,782
386,41,5460,50,1365,460,458,447,25.0,585
387,41,5465,50,1517,516,505,496,46.4,61


In [457]:
dSAT = dSAT.drop_duplicates()

In [458]:
#Next I will look for test scores outside of the possible 200-800 range
dSAT[(dSAT.MATHEMATICS<200)|(dSAT.MATHEMATICS>800)|(dSAT.CRITICAL_READING<200)|(dSAT.CRITICAL_READING>800)|(dSAT.WRITING<200)
     |(dSAT.WRITING>800)]

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
208,23,3150,300,0,0,0,0,0.0,0
209,23,3150,301,0,0,0,0,0.0,0
305,31,4010,1,0,0,0,0,0.0,0
308,31,4010,4,0,0,0,0,0.0,0
309,31,4010,5,0,0,0,0,0.0,0
362,39,4160,51,0,0,0,0,0.0,0
398,80,8010,980,0,0,0,0,0.0,0


In [459]:
# Create list of problematic County and District Codes
Co = [23,31,39,80]
Dist = [3150,4010,4160,8010]
bads = zip(Co,Dist)
bads

[(23, 3150), (31, 4010), (39, 4160), (80, 8010)]

There are 7 rows with no information other than identifying codes.  This means either that data is missing or that no one from these school took the SAT.  It seems for more likely that the SAT data just wasn't made available for these schools.  I will assign the missing school the average scores for schools in the same district, but I will leave the N_STUDENTS_SCORED field as 0 to differentiate the imputed data from real data.

In [460]:
for x,y in bads:
   
    #Find out if there is data on other schools in the same district
    length = len(dSAT.loc[(dSAT.CO_CODE==x)&(dSAT.DIST_CODE==y)])
    
    #If there is data on other schools in the district, replace missing scores with the means
    if length > 1:
        dSAT.loc[(dSAT.CO_CODE==x)&(dSAT.DIST_CODE==y)&(dSAT.MATHEMATICS==0),
        ['TOTAL','MATHEMATICS','CRITICAL_READING','WRITING','SAT_1550']] = dSAT.loc[(dSAT.CO_CODE==x)&
        (dSAT.DIST_CODE==y)&(dSAT.MATHEMATICS!=0),['TOTAL','MATHEMATICS','CRITICAL_READING','WRITING','SAT_1550']].mean().values

        #Data type changed to float by this operation. Change back to int
        dSAT.loc[:,['TOTAL','MATHEMATICS','CRITICAL_READING','WRITING']] = dSAT[['TOTAL','MATHEMATICS','CRITICAL_READING',
                                                                             'WRITING']].astype('int64')
    
    #If there are no other schools in the district drop the school from the dataframe
    else:
        k =  dSAT.loc[(dSAT.CO_CODE==x)&(dSAT.DIST_CODE==y)&(dSAT.MATHEMATICS==0)].index
        dSAT = dSAT.drop(k,axis=0)


In [461]:
#Check for SAT_1550 values that don't make sense.
dSAT[(dSAT.SAT_1550>=100)|dSAT.SAT_1550<=0]
#Only zero scores appear. These make sense due to the very low average test scores for these schools

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
83,7,680,40,1006,344,334,328,0.0,360
84,7,680,245,1030,349,337,344,0.0,192
85,7,680,305,1103,373,377,353,0.0,107
119,13,1390,80,1103,379,355,369,0.0,950
134,13,3570,70,1076,356,361,359,0.0,803
138,13,3570,305,941,293,309,339,0.0,641
306,31,4010,2,1070,366,362,342,0.0,749
307,31,4010,3,1061,369,344,348,0.0,961
311,31,4010,25,700,230,230,240,0.0,479
314,31,4010,305,1017,340,330,347,0.0,419


## Data Transformation

In [462]:
#Concatenate the 3 code columns into a single column
dSAT.loc[:,'ID'] = (dSAT.CO_CODE.astype('str') + dSAT.DIST_CODE.astype('str').str.zfill(4) + 
                    dSAT.SCH_CODE.astype('str').str.zfill(3))

In [463]:
#remove old columns
dSAT = dSAT.drop(['CO_CODE','DIST_CODE','SCH_CODE'],axis=1)

#move ID back to first column
cols = ['ID','TOTAL','MATHEMATICS','CRITICAL_READING','WRITING','SAT_1550','N_STUDENTS_SCORED']
dSAT = dSAT.loc[:,cols]

In [464]:
dSAT.head()

,ID,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
0,10110010,1320,461,429,430,23.5,88
1,10120010,1502,515,506,481,41.7,660
2,10590025,1418,486,468,464,31.1,255
3,11310005,1479,504,490,485,37.8,450
4,11790040,1477,508,486,483,35.7,235


In [465]:
de = pd.read_csv('enrollment.txt',sep='\t')
de.columns = ['ID','OVERALL_ENROL']
de.head()

,ID,OVERALL_ENROL
0,310900030,479
1,50475050,36
2,234660050,583
3,234860050,350
4,314010020,673


In [466]:
#Change ID to a string to match dSAT
de.loc[:,'ID'] = de.ID.astype('str')

In [467]:
#Some rows of the enrollment dataframe have the same ID number.  I will add these together 
de.ID.value_counts()[0:10]

411870050    2
410280020    2
414100050    2
413675050    2
415460050    2
415465050    2
215715025    1
71900050     1
806010910    1
394670050    1
Name: ID, dtype: int64

In [468]:
#Sum rows with the same ID
de = de.groupby('ID').sum()
de.reset_index(level=de.index.names, inplace=True)
de.ID.value_counts()[0:10]

350555005    1
172390050    1
133570305    1
234130050    1
131390080    1
806010910    1
411870050    1
34405050     1
292940040    1
375110010    1
Name: ID, dtype: int64

In [469]:
df = dSAT.merge(de,on='ID')

In [475]:
df.loc[:,'PERC_TAKING_SAT'] = (df.N_STUDENTS_SCORED/df.OVERALL_ENROL)*100

In [476]:
df.head()

,ID,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED,OVERALL_ENROL,PERC_TAKING_SAT
0,10110010,1320,461,429,430,23.5,88,114,77.192982
1,10120010,1502,515,506,481,41.7,660,909,72.607261
2,10590025,1418,486,468,464,31.1,255,403,63.275434
3,11310005,1479,504,490,485,37.8,450,676,66.568047
4,11790040,1477,508,486,483,35.7,235,276,85.144928


In [478]:
#check for PERC_TAKING_SAT that dont make sense
df[df.PERC_TAKING_SAT>100]

,ID,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED,OVERALL_ENROL,PERC_TAKING_SAT
13,30440020,1306,443,430,433,14.9,516,330,156.363636
20,31370040,1467,495,482,490,39.4,421,356,118.258427
24,31760050,1699,589,554,556,72.3,957,823,116.281896
43,34300050,1631,559,529,543,58.3,44,30,146.666667


There are four school where more student took the SAT than are enrolled in the school.  There must be something wrong with the data.

In [504]:
pd.melt(dSAT, id_vars=['ID'],var_name='DATA_TYPE',value_vars=['TOTAL','MATHEMATICS','CRITICAL_READING','WRITING','SAT_1550']).sort_values('ID')

,ID,DATA_TYPE,value
0,10110010,TOTAL,1320.0
1572,10110010,SAT_1550,23.5
1179,10110010,WRITING,430.0
393,10110010,MATHEMATICS,461.0
786,10110010,CRITICAL_READING,429.0
1573,10120010,SAT_1550,41.7
394,10120010,MATHEMATICS,515.0
1180,10120010,WRITING,481.0
787,10120010,CRITICAL_READING,506.0
1,10120010,TOTAL,1502.0
